In [1]:
import random
import logging
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

from sklearn  import metrics

import warnings
from sklearn.metrics import precision_score, recall_score, f1_score
# 设置警告过滤器
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
data = pd.read_csv('../../data/CMDC/phq9.csv')
data.head()

,id,label,text0,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11
0,HC01,2,食欲正常，一天三顿，然后晚上十点多的时候回宿舍会再吃点零食。,身体还行，上周有一点感冒，不过也没吃药，喝点热水就好了，,我自己的睡眠一直挺好的，我基本上躺床上就能睡，我不是那种躺床上玩手机的，我只要往床上一躺就想...,可以说是天天都交流吧，有时候会吐槽一下，实验室，几乎天天都会说一说。,记忆力还行，没有感觉到不如从前，也没感到比从前好，就是没有感觉到异样。,专注的话，感觉一般吧，因为我学习的话。基本上是坐在那里，可以学认真的学五十分钟，但是再往后面...,这个也有，就是有一些研究生阶段有一些课。然后上课没有认真听过，就觉得和自己的方向没有什么关系...,这个倒没有。我觉得生活如此美好，为什么要这样？,这个我，低落的时候倒是有，绝望还没有，就是刚毕业，那一次就是拍毕业照的时候，然后拍完之后大家...,这个多的就是，就是我，我父母生气吧，因为我妈的身体不是很好，就是可能会经常生病，然后每次打电...,这个就是？因为现在是学生阶段嘛，因为我觉得，我觉得如果真这种发生的话，主要就是有时候遇到遇到...,这个就是大部分就是身体不适的时候，或者是没有休息过来，就是午睡刚睡醒的那一阵，那个侍卫真的是...
1,HC02,1,恩上个星期六的时候发了一场高烧，恩，现在现在食欲好了，那几天就不太想吃东西。,"最近的身体状况在,就有点有点咳嗽，嘴巴里面长口疮有点不太舒服。我的，现在天天气冷，也不太喜欢...",恩晚上，我晚上睡觉的时候肯定会醒一次，每每天都是这样，每天晚上睡一半就会醒。大约两三点钟的样...,恩我平时恩在实验室的时间比较多，在宿舍就是到晚上，等我中午睡觉的时候有人回宿舍，但是那时候就...,最近记忆力，我感觉不怎么样，有时候老忘东西，有时候早上起来又不能按时起来，可能跟我之前发过烧...,恩，我之前的之前，我比较喜欢专注于一个一个事情，同时做一件事情啊。我这是我之前的专注力，是非...,恩我，我这个人感到疲倦的时候，就是中午不睡觉的时候，我中午睡觉这个习惯已经坚持了，从初中开始...,没有这个恩恩，我家里人都盼着我越来越好，之后能够找个好工作，不再像家里那样那么累那么苦，所以...,嗯。我感到绝望的一次就是在考研，在考研的面试之后的一天吧，因为我考研的初试成绩还是可以的，但...,恩有时候看到，比如说我生病了啊，在宿舍躺着啊，舍友在那还在大喊大叫的，我就感觉特别的不高兴，...,嗯。每当别人的家长都在说自己的孩子之后一年年挣多少钱啊，这个那个的时候，我就感觉到，别人是在...,恩我在人多发言的时候，就会思想比较乱，比如说之前，做思想政治的一个PPT，每个每个班每个班都...
2,HC03,2,嗯我食欲我就一直挺好就是每次跟我姐妹出去逛街你说我姐们儿嗯这其实是我姐你知道吗她也是东北的。...,嗯最近养了这只狗之后呢我就发现我也得经常遛它我也顺便他锻炼了我也锻炼了就这样了我觉得养它也有...,睡眠我觉得，嗯就最近两天晚上可能是我跟刚跟这个狗在一起玩然后有的时候儿比如说就一直有什么动静...,基本上天天交流吧，我朋友不是特别的就是不是特别多是那种比如说天天能说到一起去的可能我这人性格...,啊哦我记忆力好像一直都不太好，因为我感觉嗯从以前吗我觉得我记忆力不是特别好就是可能是我不是特...,我觉得我专注力就是没有说那种特别特别专注的那种就是比如说你现在让我看书我就看不下去然后呢，你...,嗯，比，比如说我一人我就看手机我可能看时间长了我就会疲倦，因为现在我都愁死了那为啥，我那么俩...,哦我这没有这个我主要是胆小如果你要是跳楼吧我也不敢哎我记得以前你知道吗我就我就说我我说你说我...,心情低落，沮丧绝望，我可能低落的时候我就觉得我忽然间有一天照镜子我发现哇塞我眼袋怎么这么大呢...,担心我就天天。担心我要挣钱呀，我就最近我担心的问题，焦虑，焦虑，也是挣钱，现在就主要挣钱钱挣...,那都没有我觉得我每天都进步进步自己觉得进步就行反正不管可能别人都觉得那都不重要。我妈对我以前...,嗯现在我觉得比以前慢一点儿吧嗯就是现在可能因为考虑的事比较多一些可能就是说的也多也变慢了然后...
3,HC04,1,嗯我现在等一会儿去吃饭然后现在一日三餐的话还比较正常，体重的话也在正常幅度里面波动吧。还有什...,嗯对，我经常去北外游泳然后在学校里面参加一些呃舞蹈的类似的活动啊参加我我因为我是在我们学校体...,嗯我觉得比较简单因为我们宿舍都是十一点关灯然后我早上每天早上都是七点左右起床然后每天生活都很...,嗯跟朋友交流的频率是吗？交流的话嗯跟特别好的朋友因为我的特别好的朋友在其他学院所以我们一般会...,嗯，忘事情偶尔会忘吧，不太经常有时候，就比如说小事情就是有时候可能会嗯戴着眼镜找眼镜那种其他...,历史？专注力专注力啊我做一个事情的时候就会专门做这个事情我不会去想其他事情，比如说你刚说的什...,感觉，疲倦的话我觉得不会经常发生，嗯有时候感觉疲倦的话是中午没睡午觉或者晚上吃的比较晚这会...,没有，我觉得嗯就有时候看到那些自杀的新闻我是有时候是蛮不能理解的特别是一些跳楼啊或者是怎么样...,心情低落，倒是很少心情低落。反正最近是好像是没有什么心情低落因为就,不顺心的事情，可能会有一些可能给自己定的一个目标可能没有完成或者没有办法就是完，达到那个的话...,很失败，反正近几年我市没有觉得自己很失败，虽然我自己虽然我自己成就好像也没有那么高。但是自己...,变慢我这人是一紧张说话会变快但变慢的话那可能是给一些小朋友或者一些老人讲讲事情的时候我可能语...
4,HC05,2,我食欲一直很好，没有变化挺能吃,啊我这个身体自我感觉还是比较健康的，体育锻炼这方面呃本科期间锻炼挺频繁的方式主要是。打篮球和...,最近我睡眠一直很好，\n啊我睡眠一般在8到9个小时，我一般晚上不起夜，入睡的话，就是困了就能...,我研究生。嗯，阶段来说的话基本上天天在实验室都跟师兄师弟的交流内容大部分为学术内容啊又少少部...,记忆力这方面，呃还还行吧因为现在也没有特别去关注要记的东西，所以不太好说这个不像高中你背个课...,我专注力感觉还可以有时候，就比如写个matlab matlab代码的时候有时候听歌觉得心烦就...,呃有的时候在实验室因为忙呃加班到一两点的时候回到宿舍会感觉很累。就没什么，精神头，在在就是有...,啊这个完全没有,哦这个心情不好的时候感觉还，挺。经常发生的，就有时候编程出不来或者是做的东西导师不太认可就经...,生活中的话，现在我这个年龄段儿父母都在50岁左右他们的健康问题不是很担心，然后主要是我爷爷辈...,啊失望，呃这个就是导师他对我本来对我研究生生涯有个规划。啊就，是什么阶段能干做出个啥，然后对...,说随便吧我觉得可能。就是对此对这部分内容了解不了解的时候你会花更多时间去思考所以才会导致说话...


In [3]:
datas= []
for i in range(12):
    df = data[['id','label',f'text{i}']]
    df = df.dropna()
    df = df.reset_index(drop=True)
    df = df.rename(columns={f'text{i}':'text'})
    datas.append(df)

In [4]:
from transformers import AutoTokenizer, AutoModel
import os
class CLIP(nn.Module):
    def __init__(self, text_encoder, symptom_encoder,device):  
        super(CLIP, self).__init__()
        self.device = device
        self.text_encoder = AutoModel.from_pretrained(text_encoder).to(device)
        self.symptom_encoder = AutoModel.from_pretrained(symptom_encoder).to(device)
        self.tokenizer_text = AutoTokenizer.from_pretrained(text_encoder)
        self.tokenizer_symptom = AutoTokenizer.from_pretrained(symptom_encoder)
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))
       
        
   
    def encode_text(self, text):
        inputs = self.tokenizer_text.batch_encode_plus(
            batch_text_or_text_pairs=text,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(self.text_encoder.device)

        outputs = self.text_encoder(**inputs)
        return outputs.last_hidden_state[:, 0, :].squeeze(0)
    
    def encode_symptoms(self, symptoms):
        inputs = self.tokenizer_symptom.batch_encode_plus(
            batch_text_or_text_pairs=symptoms,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(self.symptom_encoder.device)

        outputs = self.symptom_encoder(**inputs)
        return outputs.last_hidden_state[:, 0, :].squeeze(0)
    def forward(self, text, symptoms):
        text_features = self.encode_text(text)
        symptom_features = self.encode_symptoms(symptoms)
         # Normalize features
        text_features = F.normalize(text_features, dim=-1)
        symptom_features = F.normalize(symptom_features, dim=-1)
        return text_features, symptom_features, self.logit_scale.exp()
        
    def get_similarity(self, text, symptoms):
        
        text_features = self.encode_text(text)
        symptom_features = self.encode_symptoms(symptoms)
        
        # Normalize features
        text_features = F.normalize(text_features, dim=-1)
        symptom_features = F.normalize(symptom_features, dim=-1)

        # Compute logits
        logit_scale = self.logit_scale.exp()
        logit_per_text = logit_scale * text_features @ symptom_features.t()
        logit_per_symptom = logit_scale * symptom_features @ text_features.t()

        return logit_per_text, logit_per_symptom
    

In [5]:
warnings.filterwarnings("ignore", category=UserWarning, message="TypedStorage is deprecated.*")
def rep_clip_extract(device, sents, labels):
    # 加载模型
    clip = CLIP("../../../roberta_chinese", "../../../bert-base-chinese", device)
    clip.load_state_dict(torch.load("../../models/add_swdd_normal_clip.pth"))
    clip.eval()
    sents_reps = []
    step = 16
    for idx in range(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)        
        sents_batch = sents[idx: idx_end]
        # print(sents_batch)
        
        with torch.no_grad():
            text_features = clip.encode_text(sents_batch)
            
        sents_reps.append(text_features.cpu())
        
    sents_reps = torch.cat(sents_reps)
    for idx in range(len(labels)):
        labels[idx] = torch.tensor(labels[idx])
    labels = torch.stack(labels)

    print(sents_reps.shape)
    print(labels.shape)
    return sents_reps,labels    
    # path = f'./{path}add_swdd_mulsym_clip_tensor/'
    # if not os.path.exists(path):
    #     os.makedirs(path)
    # torch.save(sents_reps, path + 'sents.pt')
    # torch.save(labels, path + 'labels.pt')

    

In [6]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
clips = []
for i in range(12):
    clips.append(rep_clip_extract(device, datas[i]['text'].tolist(), datas[i]['label'].tolist()))

torch.Size([78, 768])
torch.Size([78])
torch.Size([78, 768])
torch.Size([78])
torch.Size([78, 768])
torch.Size([78])
torch.Size([78, 768])
torch.Size([78])
torch.Size([78, 768])
torch.Size([78])
torch.Size([77, 768])
torch.Size([77])
torch.Size([78, 768])
torch.Size([78])
torch.Size([78, 768])
torch.Size([78])
torch.Size([77, 768])
torch.Size([77])
torch.Size([74, 768])
torch.Size([74])
torch.Size([74, 768])
torch.Size([74])
torch.Size([76, 768])
torch.Size([76])


In [7]:
berts = []
for i in range(12):
    berts.append(torch.load(f'./bert_tensor/res{i}/sents.pt'))

In [8]:
#构建dataset
class MyDataset(Dataset):
    def __init__(self, clip_data, bert_features):
        self.clip_features = clip_data[0]
        self.bert_features = bert_features
        self.labels = clip_data[1]
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return self.clip_features[idx], self.bert_features[idx], self.labels[idx]
        

In [9]:
datasets = []
for i in range(12):
    datasets.append(MyDataset(clips[i], berts[i]))

In [10]:
class mymodel(nn.Module):
    def __init__(self, q_dim, kv_dim, output_size):
        super(mymodel, self).__init__()
        
        # 特征变换层
        self.q_transform = nn.Linear(q_dim, 64)
        self.k_transform = nn.Linear(kv_dim, 64)
        self.v_transform = nn.Linear(kv_dim, 64)
        
        self.q1_transform = nn.Linear(q_dim, 64)
        self.k1_transform = nn.Linear(kv_dim, 64)
        self.v1_transform = nn.Linear(kv_dim, 64)
        # 注意力层
        self.attention = nn.MultiheadAttention(embed_dim=64, num_heads=4)
        self.attention1 = nn.MultiheadAttention(embed_dim=64, num_heads=4)
        self.m = torch.nn.Parameter(torch.tensor(0.5))
        # 融合层
        self.fusion = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, output_size)
        )
    
    def forward(self, q, k):
        # 特征变换
        q_transformed = self.q_transform(q)
        k_transformed = self.k_transform(k)
        v_transformed = self.v_transform(k)
        
        q1_transformed = self.q1_transform(k)
        k1_transformed = self.k1_transform(q)
        v1_transformed = self.v1_transform(q)
        # 交叉注意力融合
        f1, _ = self.attention(q_transformed, k_transformed, v_transformed)
        f2, _ = self.attention1(q1_transformed, k1_transformed, v1_transformed)
        fused_feature = self.m*f1 + (1-self.m)*f2
        
        # 最终融合
        output = self.fusion(fused_feature)
        
        return output

In [11]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
import sys

def Train(dataloader, device, model, loss_fn, optimizer, logger):
    loss_list = []
    y_true_list = []
    y_pred_list = []

    for batch_i, batch_loader in enumerate(dataloader):
        batch_x1,batch_x2, batch_y = batch_loader
        batch_x1 = batch_x1.to(device)
        batch_x2 = batch_x2.to(device)
        batch_y = batch_y.clone().detach().float()  # 不需要转换为长整型，因为是回归任务
        batch_y = batch_y.to(device).unsqueeze(1)

        model.train()
        pred = model(batch_x1.float(),batch_x2.float())
        
        loss = loss_fn(pred, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(batch_y)
        # print(pred)
 
        # 将预测值和真实值存储以便计算指标
        y_true_list.extend(batch_y.cpu().numpy())
        y_pred_list.extend(pred.cpu().detach().numpy())

        loss_list.append(loss.item())

    
    # 计算 MAE 和 RMSE
    mae = mean_absolute_error(y_true_list, y_pred_list)
    rmse = np.sqrt(mean_squared_error(y_true_list, y_pred_list))

    logger.info(f'Loss: {np.mean(loss_list):.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')

In [12]:
def Test(dataloader, device, model, loss_fn, logger):
    
    avg_loss = 0
    total_pred, total_y = [], []

    for batch_i, batch_loader in enumerate(dataloader):
        batch_x1,batch_x2, batch_y = batch_loader
        batch_x1 = batch_x1.to(device)
        batch_x2 = batch_x2.to(device)
        batch_y = batch_y.clone().detach().float()  # 不需要转换为长整型，因为是回归任务
        batch_y = batch_y.to(device).unsqueeze(1)

        model.eval()
        with torch.no_grad():
            pred = model(batch_x1.float(),batch_x2.float())
            loss = loss_fn(pred, batch_y)
            avg_loss += loss.item()

        total_pred.append(pred.cpu())
        total_y.append(batch_y.cpu())
    
    avg_loss = avg_loss / (batch_i + 1)
    
    total_y = torch.cat(total_y)
    total_pred = torch.cat(total_pred)

    # 计算 MAE 和 RMSE
    mae = mean_absolute_error(total_y.numpy(), total_pred.numpy())
    rmse = np.sqrt(mean_squared_error(total_y.numpy(), total_pred.numpy()))

    logger.info(f'Avg loss: {avg_loss:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')
    return avg_loss, mae, rmse,total_pred.numpy()

In [13]:
from torch.utils.data import Subset


def Start_train(k, dataset, learning_rate, batch_size, criterion, epoches,log_path,device,y_pred_list):

    kfold_mae_list = []
    kfold_rmse_list = []
    kfold_loss_list = []
    

    
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)

    # log_path = './logging/add_swdd_mulsym_clip.log'
    if not os.path.exists(os.path.dirname(log_path)):
        os.makedirs(os.path.dirname(log_path))
    file_handler = logging.FileHandler(log_path)
    
    # 配置文件处理器的格式
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    
    # 添加文件处理器到日志记录器
    logger.addHandler(file_handler)

    # device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    fold_size = len(dataset) // k  # 每个折的大小

#    创建K个折的训练集和验证集
    train_datasets = []
    val_datasets = []
    
    for i in range(k):
        val_indices = range(i * fold_size, (i + 1) * fold_size)
        train_indices = [idx for idx in range(len(dataset)) if idx not in val_indices]
    
        train_subset = Subset(dataset, train_indices)
        val_subset = Subset(dataset, val_indices)
    
        train_datasets.append(train_subset)
        val_datasets.append(val_subset)
    seed = 42
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    
    for fold in range(k):
        # 创建新的模型实例
        logger.info(f'--------------------------- fold {fold} ---------')
        model = mymodel(768, 768,1).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
        # 创建训练集和验证集的数据加载器
        train_loader = DataLoader(train_datasets[fold], batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_datasets[fold], batch_size=batch_size)
        mae_best = np.inf
        rmse_best = np.inf
        loss_best = 0
        y_pred_best = []

        
        for epoch in range(epoches):
            logger.info(f'--------------------------- epoch {epoch} ---------------------------')
            logger.info('training ...')
            Train(train_loader, device, model, criterion, optimizer, logger)
            logger.info("                                                                         ")
            logger.info('evaluate ...')
            loss, mae, rmse,  y_pred = Test(val_loader, device, model, criterion, logger)
            if mae < mae_best:
                mae_best = mae
                rmse_best = rmse
                loss_best = loss
                y_pred_best = y_pred
  
        kfold_mae_list.append(mae_best)
        kfold_rmse_list.append(rmse_best)
        kfold_loss_list.append(loss_best)
        y_pred_list.append(y_pred_best)
    
    logger.info("                                                               ")
    logger.info(f'K-fold loss: {np.mean(kfold_loss_list):.4f}')
    logger.info(f'K-fold mae: {np.mean(kfold_mae_list):.4f}')
    logger.info(f'K-fold rmse: {np.mean(kfold_rmse_list):.4f}')


    # return kfold_acc_list, kfold_loss_list, kfold_precision_list, kfold_recall_list, kfold_f1_list

In [14]:
learning_rate = 0.01
batch_size = 8
criterion = nn.MSELoss()
epoches = 50
k = 10

In [34]:
log_path = './logging/dualcross_q1.log'
Start_train(k, datasets[0], learning_rate, batch_size, criterion, epoches,log_path,device)

2025-07-07 17:25:00,867 - INFO - --------------------------- fold 0 ---------
2025-07-07 17:25:00,895 - INFO - --------------------------- epoch 0 ---------------------------
2025-07-07 17:25:00,896 - INFO - training ...
2025-07-07 17:25:00,963 - INFO - Loss: 85.1446, MAE: 7.0804, RMSE: 9.2639
2025-07-07 17:25:00,964 - INFO -                                                                          
2025-07-07 17:25:00,965 - INFO - evaluate ...
2025-07-07 17:25:00,970 - INFO - Avg loss: 14.9419, MAE: 3.8390, RMSE: 3.8655
2025-07-07 17:25:00,970 - INFO - --------------------------- epoch 1 ---------------------------
2025-07-07 17:25:00,971 - INFO - training ...
2025-07-07 17:25:01,039 - INFO - Loss: 65.6786, MAE: 6.8091, RMSE: 8.0990
2025-07-07 17:25:01,040 - INFO -                                                                          
2025-07-07 17:25:01,041 - INFO - evaluate ...
2025-07-07 17:25:01,046 - INFO - Avg loss: 11.7326, MAE: 3.3954, RMSE: 3.4253
2025-07-07 17:25:01,047 - 

In [36]:
import numpy

p = np.concatenate(y_pred_list, axis=0)

for i in range(len(p)):
    print(datasets[0][i][2],p[i])
63

tensor(2) [1.9974325]
tensor(1) [1.9973915]
tensor(2) [1.9974325]
tensor(1) [1.9974325]
tensor(2) [1.9969589]
tensor(2) [1.9973464]
tensor(2) [1.9974325]
tensor(1) [1.9587089]
tensor(5) [1.9587089]
tensor(0) [1.9587089]
tensor(2) [1.9587089]
tensor(0) [1.9587089]
tensor(5) [1.9587089]
tensor(3) [1.9587089]
tensor(2) [1.9563651]
tensor(5) [1.9563651]
tensor(3) [1.9563651]
tensor(0) [1.9563651]
tensor(2) [1.9563651]
tensor(3) [1.9563651]
tensor(0) [1.9563651]
tensor(0) [2.0357637]
tensor(2) [1.974488]
tensor(3) [2.0357637]
tensor(5) [2.0357637]
tensor(1) [2.0357637]
tensor(2) [2.0357637]
tensor(3) [1.987603]
tensor(0) [2.5971506]
tensor(0) [2.5951388]
tensor(0) [2.5942943]
tensor(0) [2.594159]
tensor(0) [2.5967917]
tensor(4) [2.5949724]
tensor(1) [2.5917246]
tensor(0) [0.54448515]
tensor(1) [0.5444641]
tensor(1) [0.5441451]
tensor(0) [0.5444502]
tensor(0) [0.54424095]
tensor(0) [0.5443783]
tensor(0) [0.54421145]
tensor(2) [1.6433759]
tensor(0) [1.6433759]
tensor(0) [1.6433759]
tensor(0) 

In [38]:
log_path = './logging/dualcross_q2.log'
y_pred_list = []
Start_train(k, datasets[1], learning_rate, batch_size, criterion, epoches,log_path,device,y_pred_list)

2025-07-07 17:39:19,064 - INFO - --------------------------- fold 0 ---------
2025-07-07 17:39:19,092 - INFO - --------------------------- epoch 0 ---------------------------
2025-07-07 17:39:19,092 - INFO - training ...
2025-07-07 17:39:19,167 - INFO - Loss: 82.8669, MAE: 7.3511, RMSE: 9.1387
2025-07-07 17:39:19,169 - INFO -                                                                          
2025-07-07 17:39:19,169 - INFO - evaluate ...
2025-07-07 17:39:19,175 - INFO - Avg loss: 3.3684, MAE: 1.7669, RMSE: 1.8353
2025-07-07 17:39:19,176 - INFO - --------------------------- epoch 1 ---------------------------
2025-07-07 17:39:19,177 - INFO - training ...
2025-07-07 17:39:19,245 - INFO - Loss: 69.2982, MAE: 7.1906, RMSE: 8.3244
2025-07-07 17:39:19,247 - INFO -                                                                          
2025-07-07 17:39:19,248 - INFO - evaluate ...
2025-07-07 17:39:19,253 - INFO - Avg loss: 8.6410, MAE: 2.8836, RMSE: 2.9396
2025-07-07 17:39:19,254 - IN

In [40]:
p = np.concatenate(y_pred_list, axis=0)
for i in range(len(p)):
    print(datasets[1][i][2],p[i])

tensor(2) [2.0023048]
tensor(1) [2.0023048]
tensor(2) [2.0023048]
tensor(1) [2.0023048]
tensor(2) [2.0023048]
tensor(2) [2.0023048]
tensor(2) [2.0023048]
tensor(1) [1.5464056]
tensor(5) [1.5464056]
tensor(0) [1.5464056]
tensor(2) [1.5464056]
tensor(0) [1.5464056]
tensor(5) [1.5464056]
tensor(3) [1.5464056]
tensor(2) [2.3655267]
tensor(5) [2.3123033]
tensor(3) [2.3449855]
tensor(0) [2.3649397]
tensor(2) [2.2928543]
tensor(3) [2.3657973]
tensor(0) [2.2946484]
tensor(0) [2.061998]
tensor(2) [2.061998]
tensor(3) [2.061998]
tensor(5) [2.061998]
tensor(1) [2.061998]
tensor(2) [2.061998]
tensor(3) [2.061998]
tensor(0) [1.9620779]
tensor(0) [1.4194683]
tensor(0) [1.4501371]
tensor(0) [1.8135225]
tensor(0) [1.9620779]
tensor(4) [1.9620779]
tensor(1) [1.8214651]
tensor(0) [1.3512316]
tensor(1) [1.3512316]
tensor(1) [1.3512316]
tensor(0) [1.3512316]
tensor(0) [1.3512316]
tensor(0) [1.3512316]
tensor(0) [1.3512316]
tensor(2) [1.3437833]
tensor(0) [1.3437833]
tensor(0) [1.3437833]
tensor(0) [1.3437

In [45]:
log_path = './logging/dualcross_q3.log'
y_pred_list = []
Start_train(k, datasets[2], learning_rate, batch_size, criterion, epoches,log_path,device,y_pred_list)

2025-07-07 20:32:54,006 - INFO - --------------------------- fold 0 ---------
2025-07-07 20:32:54,031 - INFO - --------------------------- epoch 0 ---------------------------
2025-07-07 20:32:54,032 - INFO - training ...
2025-07-07 20:32:54,085 - INFO - Loss: 80.3600, MAE: 7.2507, RMSE: 8.9937
2025-07-07 20:32:54,087 - INFO -                                                                          
2025-07-07 20:32:54,087 - INFO - evaluate ...
2025-07-07 20:32:54,092 - INFO - Avg loss: 21.4785, MAE: 4.6113, RMSE: 4.6345
2025-07-07 20:32:54,093 - INFO - --------------------------- epoch 1 ---------------------------
2025-07-07 20:32:54,094 - INFO - training ...
2025-07-07 20:32:54,134 - INFO - Loss: 69.9368, MAE: 6.8881, RMSE: 8.3596
2025-07-07 20:32:54,135 - INFO -                                                                          
2025-07-07 20:32:54,136 - INFO - evaluate ...
2025-07-07 20:32:54,141 - INFO - Avg loss: 31.0431, MAE: 5.5523, RMSE: 5.5716
2025-07-07 20:32:54,142 - 

In [46]:
p = np.concatenate(y_pred_list, axis=0)
for i in range(len(p)):
    print(datasets[2][i][2],p[i])

tensor(2) [1.8079318]
tensor(1) [1.8079318]
tensor(2) [1.8079318]
tensor(1) [1.8079318]
tensor(2) [1.8079318]
tensor(2) [1.8079318]
tensor(2) [1.8079318]
tensor(1) [1.9751799]
tensor(5) [1.9751799]
tensor(0) [1.9751799]
tensor(2) [1.9751799]
tensor(0) [1.9751799]
tensor(5) [1.9751799]
tensor(3) [1.9751799]
tensor(2) [2.134061]
tensor(5) [2.157642]
tensor(3) [2.134061]
tensor(0) [2.1406212]
tensor(2) [2.134061]
tensor(3) [2.134076]
tensor(0) [2.134061]
tensor(0) [2.270772]
tensor(2) [2.2707725]
tensor(3) [2.270772]
tensor(5) [2.4037592]
tensor(1) [2.5038981]
tensor(2) [2.270772]
tensor(3) [2.270772]
tensor(0) [1.2915673]
tensor(0) [1.2915673]
tensor(0) [1.2915673]
tensor(0) [1.2915673]
tensor(0) [1.2915673]
tensor(4) [1.2915673]
tensor(1) [1.2915673]
tensor(0) [0.5207766]
tensor(1) [0.5207766]
tensor(1) [0.5207766]
tensor(0) [0.5207766]
tensor(0) [0.5207766]
tensor(0) [0.5207766]
tensor(0) [0.5207766]
tensor(2) [1.2323836]
tensor(0) [1.2323836]
tensor(0) [1.2323836]
tensor(0) [1.2323836

In [24]:
log_path = './logging/dualcross_q4.log'
Start_train(k, datasets[3], learning_rate, batch_size, criterion, epoches,log_path,device)

2025-07-07 16:40:44,323 - INFO - --------------------------- fold 0 ---------
2025-07-07 16:40:44,373 - INFO - --------------------------- epoch 0 ---------------------------
2025-07-07 16:40:44,374 - INFO - training ...
2025-07-07 16:40:44,446 - INFO - Loss: 104.5122, MAE: 6.7138, RMSE: 10.2586
2025-07-07 16:40:44,448 - INFO -                                                                          
2025-07-07 16:40:44,449 - INFO - evaluate ...
2025-07-07 16:40:44,454 - INFO - Avg loss: 0.6418, MAE: 0.6610, RMSE: 0.8011
2025-07-07 16:40:44,455 - INFO - --------------------------- epoch 1 ---------------------------
2025-07-07 16:40:44,455 - INFO - training ...
2025-07-07 16:40:44,518 - INFO - Loss: 73.9501, MAE: 7.2789, RMSE: 8.5970
2025-07-07 16:40:44,519 - INFO -                                                                          
2025-07-07 16:40:44,520 - INFO - evaluate ...
2025-07-07 16:40:44,524 - INFO - Avg loss: 66.4794, MAE: 8.1406, RMSE: 8.1535
2025-07-07 16:40:44,525 -

In [32]:
log_path = './logging/dualcross_q5.log'
Start_train(k, datasets[4], learning_rate, batch_size, criterion, epoches,log_path,device)

2025-05-07 10:58:49,464 - INFO - --------------------------- fold 0 ---------
2025-05-07 10:58:49,474 - INFO - --------------------------- epoch 0 ---------------------------
2025-05-07 10:58:49,475 - INFO - training ...
2025-05-07 10:58:49,536 - INFO - Loss: 82.6775, MAE: 7.2398, RMSE: 9.1268
2025-05-07 10:58:49,538 - INFO -                                                                          
2025-05-07 10:58:49,539 - INFO - evaluate ...
2025-05-07 10:58:49,543 - INFO - Avg loss: 45.8700, MAE: 6.7561, RMSE: 6.7727
2025-05-07 10:58:49,544 - INFO - --------------------------- epoch 1 ---------------------------
2025-05-07 10:58:49,545 - INFO - training ...
2025-05-07 10:58:49,601 - INFO - Loss: 66.2766, MAE: 6.9608, RMSE: 8.1350
2025-05-07 10:58:49,602 - INFO -                                                                          
2025-05-07 10:58:49,603 - INFO - evaluate ...
2025-05-07 10:58:49,607 - INFO - Avg loss: 19.4286, MAE: 4.3831, RMSE: 4.4078
2025-05-07 10:58:49,609 - 

In [33]:
log_path = './logging/dualcross_q6.log'
Start_train(k, datasets[5], learning_rate, batch_size, criterion, epoches,log_path,device)

2025-05-07 10:59:51,304 - INFO - --------------------------- fold 0 ---------
2025-05-07 10:59:51,313 - INFO - --------------------------- epoch 0 ---------------------------
2025-05-07 10:59:51,314 - INFO - training ...
2025-05-07 10:59:51,373 - INFO - Loss: 114.8034, MAE: 7.6447, RMSE: 10.7551
2025-05-07 10:59:51,375 - INFO -                                                                          
2025-05-07 10:59:51,375 - INFO - evaluate ...
2025-05-07 10:59:51,380 - INFO - Avg loss: 1.9196, MAE: 1.3167, RMSE: 1.3855
2025-05-07 10:59:51,381 - INFO - --------------------------- epoch 1 ---------------------------
2025-05-07 10:59:51,381 - INFO - training ...
2025-05-07 10:59:51,439 - INFO - Loss: 82.5443, MAE: 6.8266, RMSE: 8.9386
2025-05-07 10:59:51,440 - INFO -                                                                          
2025-05-07 10:59:51,441 - INFO - evaluate ...
2025-05-07 10:59:51,445 - INFO - Avg loss: 1.3691, MAE: 1.0732, RMSE: 1.1701
2025-05-07 10:59:51,446 - 

In [47]:
log_path = './logging/dualcross_q7.log'
y_pred_list = []
Start_train(k, datasets[6], learning_rate, batch_size, criterion, epoches,log_path,device,y_pred_list)

2025-07-07 20:37:01,222 - INFO - --------------------------- fold 0 ---------
2025-07-07 20:37:01,269 - INFO - --------------------------- epoch 0 ---------------------------
2025-07-07 20:37:01,270 - INFO - training ...
2025-07-07 20:37:01,337 - INFO - Loss: 86.6801, MAE: 7.1315, RMSE: 9.3479
2025-07-07 20:37:01,339 - INFO -                                                                          
2025-07-07 20:37:01,340 - INFO - evaluate ...
2025-07-07 20:37:01,344 - INFO - Avg loss: 17.0903, MAE: 4.1002, RMSE: 4.1340
2025-07-07 20:37:01,346 - INFO - --------------------------- epoch 1 ---------------------------
2025-07-07 20:37:01,346 - INFO - training ...
2025-07-07 20:37:01,402 - INFO - Loss: 64.5960, MAE: 6.9328, RMSE: 8.0263
2025-07-07 20:37:01,404 - INFO -                                                                          
2025-07-07 20:37:01,405 - INFO - evaluate ...
2025-07-07 20:37:01,409 - INFO - Avg loss: 10.9352, MAE: 3.2757, RMSE: 3.3068
2025-07-07 20:37:01,410 - 

In [48]:
p = np.concatenate(y_pred_list, axis=0)
for i in range(len(p)):
    print(datasets[6][i][2],p[i])

tensor(2) [2.037055]
tensor(1) [2.0370593]
tensor(2) [2.0371206]
tensor(1) [2.0370972]
tensor(2) [2.037055]
tensor(2) [2.037055]
tensor(2) [2.037055]
tensor(1) [1.9859049]
tensor(5) [1.9859049]
tensor(0) [1.9859049]
tensor(2) [1.9859049]
tensor(0) [1.9859049]
tensor(5) [1.9859049]
tensor(3) [1.9859049]
tensor(2) [2.2015164]
tensor(5) [2.2015164]
tensor(3) [2.2015164]
tensor(0) [2.2015164]
tensor(2) [2.2015164]
tensor(3) [2.2015164]
tensor(0) [2.2015164]
tensor(0) [2.1295838]
tensor(2) [2.1295838]
tensor(3) [2.1210399]
tensor(5) [2.2020297]
tensor(1) [2.1565707]
tensor(2) [2.1295838]
tensor(3) [2.2020297]
tensor(0) [2.8432999]
tensor(0) [2.8432999]
tensor(0) [2.8432999]
tensor(0) [2.8433356]
tensor(0) [2.8486]
tensor(4) [2.8432999]
tensor(1) [2.8485842]
tensor(0) [0.7310567]
tensor(1) [0.7310567]
tensor(1) [0.7310567]
tensor(0) [0.7310567]
tensor(0) [0.7310567]
tensor(0) [0.7310567]
tensor(0) [0.7310567]
tensor(2) [2.17425]
tensor(0) [2.17425]
tensor(0) [2.17425]
tensor(0) [2.17425]
ten

In [15]:
log_path = './logging/dualcross_q8.log'
y_pred_list = []
Start_train(k, datasets[7], learning_rate, batch_size, criterion, epoches,log_path,device,y_pred_list)

2025-07-10 19:45:10,274 - INFO - --------------------------- fold 0 ---------
2025-07-10 19:45:10,319 - INFO - --------------------------- epoch 0 ---------------------------
2025-07-10 19:45:10,320 - INFO - training ...
2025-07-10 19:45:10,664 - INFO - Loss: 87.1520, MAE: 7.1619, RMSE: 9.3737
2025-07-10 19:45:10,666 - INFO -                                                                          
2025-07-10 19:45:10,667 - INFO - evaluate ...
2025-07-10 19:45:10,672 - INFO - Avg loss: 28.0238, MAE: 5.2822, RMSE: 5.2938
2025-07-10 19:45:10,672 - INFO - --------------------------- epoch 1 ---------------------------
2025-07-10 19:45:10,673 - INFO - training ...
2025-07-10 19:45:10,740 - INFO - Loss: 63.6530, MAE: 6.7110, RMSE: 7.9712
2025-07-10 19:45:10,742 - INFO -                                                                          
2025-07-10 19:45:10,743 - INFO - evaluate ...
2025-07-10 19:45:10,749 - INFO - Avg loss: 13.3017, MAE: 3.5606, RMSE: 3.6472
2025-07-10 19:45:10,750 - 

In [17]:
p = np.concatenate(y_pred_list, axis=0)
for i in range(len(p)):
    print(datasets[7][i][2],p[i])

tensor(2) [1.4534984]
tensor(1) [1.4534982]
tensor(2) [1.4534984]
tensor(1) [1.7658787]
tensor(2) [1.4863995]
tensor(2) [1.4863995]
tensor(2) [1.4863995]
tensor(1) [2.1529765]
tensor(5) [2.1529765]
tensor(0) [2.1529765]
tensor(2) [2.1529765]
tensor(0) [2.1529765]
tensor(5) [2.1529765]
tensor(3) [2.1529765]
tensor(2) [2.444896]
tensor(5) [2.444896]
tensor(3) [2.444896]
tensor(0) [2.444896]
tensor(2) [2.444896]
tensor(3) [2.444896]
tensor(0) [2.444896]
tensor(0) [1.5679315]
tensor(2) [1.5679315]
tensor(3) [1.5679315]
tensor(5) [1.5679315]
tensor(1) [1.5679315]
tensor(2) [1.5679315]
tensor(3) [2.093048]
tensor(0) [1.8406837]
tensor(0) [1.8406837]
tensor(0) [0.52757955]
tensor(0) [0.6007596]
tensor(0) [0.66204226]
tensor(4) [0.6620049]
tensor(1) [0.56304336]
tensor(0) [2.4160504]
tensor(1) [2.5848694]
tensor(1) [2.4160504]
tensor(0) [2.5848694]
tensor(0) [2.5848694]
tensor(0) [2.5848694]
tensor(0) [2.4160504]
tensor(2) [0.9798374]
tensor(0) [0.9798374]
tensor(0) [0.9798374]
tensor(0) [0.97

In [36]:
log_path = './logging/dualcross_q9.log'
Start_train(k, datasets[8], learning_rate, batch_size, criterion, epoches,log_path,device)

2025-05-07 11:08:23,618 - INFO - --------------------------- fold 0 ---------
2025-05-07 11:08:23,672 - INFO - --------------------------- epoch 0 ---------------------------
2025-05-07 11:08:23,675 - INFO - training ...
2025-05-07 11:08:23,805 - INFO - Loss: 89.4425, MAE: 7.2431, RMSE: 9.5044
2025-05-07 11:08:23,807 - INFO -                                                                          
2025-05-07 11:08:23,807 - INFO - evaluate ...
2025-05-07 11:08:23,821 - INFO - Avg loss: 16.7714, MAE: 4.0751, RMSE: 4.0953
2025-05-07 11:08:23,822 - INFO - --------------------------- epoch 1 ---------------------------
2025-05-07 11:08:23,823 - INFO - training ...
2025-05-07 11:08:23,933 - INFO - Loss: 71.5402, MAE: 7.0990, RMSE: 8.4118
2025-05-07 11:08:23,935 - INFO -                                                                          
2025-05-07 11:08:23,935 - INFO - evaluate ...
2025-05-07 11:08:23,940 - INFO - Avg loss: 2.0429, MAE: 1.3579, RMSE: 1.4293
2025-05-07 11:08:23,941 - I

In [37]:
log_path = './logging/dualcross_q10.log'
Start_train(k, datasets[9], learning_rate, batch_size, criterion, epoches,log_path,device)

2025-05-07 11:20:31,808 - INFO - --------------------------- fold 0 ---------
2025-05-07 11:20:31,819 - INFO - --------------------------- epoch 0 ---------------------------
2025-05-07 11:20:31,820 - INFO - training ...
2025-05-07 11:20:31,879 - INFO - Loss: 97.3852, MAE: 7.6340, RMSE: 9.7083
2025-05-07 11:20:31,880 - INFO -                                                                          
2025-05-07 11:20:31,881 - INFO - evaluate ...
2025-05-07 11:20:31,885 - INFO - Avg loss: 57.4915, MAE: 7.5688, RMSE: 7.5823
2025-05-07 11:20:31,886 - INFO - --------------------------- epoch 1 ---------------------------
2025-05-07 11:20:31,887 - INFO - training ...
2025-05-07 11:20:31,943 - INFO - Loss: 82.6522, MAE: 6.4488, RMSE: 8.6985
2025-05-07 11:20:31,945 - INFO -                                                                          
2025-05-07 11:20:31,945 - INFO - evaluate ...
2025-05-07 11:20:31,950 - INFO - Avg loss: 102.1831, MAE: 10.0985, RMSE: 10.1086
2025-05-07 11:20:31,951

In [38]:
log_path = './logging/dualcross_q11.log'
Start_train(k, datasets[10], learning_rate, batch_size, criterion, epoches,log_path,device)

2025-05-07 11:25:34,008 - INFO - --------------------------- fold 0 ---------
2025-05-07 11:25:34,020 - INFO - --------------------------- epoch 0 ---------------------------
2025-05-07 11:25:34,021 - INFO - training ...
2025-05-07 11:25:34,079 - INFO - Loss: 98.7478, MAE: 7.2174, RMSE: 9.5774
2025-05-07 11:25:34,081 - INFO -                                                                          
2025-05-07 11:25:34,081 - INFO - evaluate ...
2025-05-07 11:25:34,086 - INFO - Avg loss: 46.8543, MAE: 6.8298, RMSE: 6.8450
2025-05-07 11:25:34,087 - INFO - --------------------------- epoch 1 ---------------------------
2025-05-07 11:25:34,088 - INFO - training ...
2025-05-07 11:25:34,145 - INFO - Loss: 75.4326, MAE: 6.2959, RMSE: 8.6293
2025-05-07 11:25:34,147 - INFO -                                                                          
2025-05-07 11:25:34,148 - INFO - evaluate ...
2025-05-07 11:25:34,152 - INFO - Avg loss: 48.4874, MAE: 6.9486, RMSE: 6.9633
2025-05-07 11:25:34,153 - 

In [39]:
log_path = './logging/dualcross_q12.log'
Start_train(k, datasets[11], learning_rate, batch_size, criterion, epoches,log_path,device)

2025-05-07 11:27:08,304 - INFO - --------------------------- fold 0 ---------
2025-05-07 11:27:08,354 - INFO - --------------------------- epoch 0 ---------------------------
2025-05-07 11:27:08,356 - INFO - training ...
2025-05-07 11:27:08,486 - INFO - Loss: 91.9932, MAE: 7.0535, RMSE: 9.5889
2025-05-07 11:27:08,487 - INFO -                                                                          
2025-05-07 11:27:08,489 - INFO - evaluate ...
2025-05-07 11:27:08,494 - INFO - Avg loss: 44.4536, MAE: 6.6518, RMSE: 6.6674
2025-05-07 11:27:08,495 - INFO - --------------------------- epoch 1 ---------------------------
2025-05-07 11:27:08,497 - INFO - training ...
2025-05-07 11:27:08,555 - INFO - Loss: 75.9802, MAE: 7.4548, RMSE: 8.6351
2025-05-07 11:27:08,557 - INFO -                                                                          
2025-05-07 11:27:08,558 - INFO - evaluate ...
2025-05-07 11:27:08,564 - INFO - Avg loss: 13.5339, MAE: 3.6568, RMSE: 3.6788
2025-05-07 11:27:08,565 - 